In [1]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
import gc

# Using Cobrapy 0.13.0
import cobra
import cobra.test
import cobra.flux_analysis.gapfilling
from cobra import Reaction, Metabolite
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
from cobra.util import solver as sutil


#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

In [2]:
# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [3]:
#Set media

def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-20.


In [6]:
# Read in  model
model=read_model(cwd+'/Gc_GENRE_2022/Models/AE004969.1.json')
model.objective='Nm_Ess_biomass'
model

Name,AE004969.1
Memory address,0x020431c76280
Number of metabolites,1297
Number of reactions,1484
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,Cytosol


In [9]:
complete(model)
print ('Complete Media:', model.slim_optimize())

Complete Media: 4.216063599310883


In [16]:
# Fix compartment labels to be more specific  

In [10]:
for metabolite in model.metabolites:
    if '_c' in metabolite.id:
        metabolite.compartment = 'cytosol'
    if '_p' in metabolite.id:
        metabolite.compartment = 'periplasm'
    if '_e' in metabolite.id:
        metabolite.compartment = 'extracellular'

In [12]:
model

Name,AE004969.1
Memory address,0x020431c76280
Number of metabolites,1297
Number of reactions,1484
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular, Cytosol"


In [13]:
for metabolite in model.metabolites:
    if 'Cytosol' in metabolite.compartment:
        print (metabolite.id)
        metabolite.compartment = 'cytosol'

essbiomass
protein
capminusbiomass
biomass


In [14]:
model

Name,AE004969.1
Memory address,0x020431c76280
Number of metabolites,1297
Number of reactions,1484
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular"


In [15]:
len(model.genes)

514

In [16]:
# Set up counts for tracking changes
init_rxn = [str(x.id) for x in model.reactions]

new_genes = 0
new_rxns = 0
new_cpds = 0

In [19]:
#Count number of imbalances
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

425 425 425


,Reactions,Charge,Masses
0,3HAD161x,,"{'H': 96.0, 'O': 17.0, 'Ab': 1.0, 'A': 2.0, 'C..."
1,3OAR161x,,"{'C': -27.0, 'H': -49.0, 'N': -2.0, 'O': -9.0,..."
2,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
3,ACCOAC,,"{'C': -9.999999999621423e-06, 'H': -1.49999999..."
4,ACHBS,,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
...,...,...,...
420,THRD_L,,"{'C': -8.000000000230045e-07, 'H': -8.00000000..."
421,THRS,,"{'H': -8.000000004670937e-07, 'O': -5.99999999..."
422,TRDR1,,{'X': 1.0}
423,TSULST,,{'H': -2.0}


In [20]:
#Several imbalances are due to missing formulas. 
#Find missing formulas
no_formula = ("")
for metabolites in model.metabolites:
        if metabolites.formula == no_formula:
            print (metabolites.id)

ps141b_c
ps161_c
essbiomass
pe160b_c
pe160b_p
Hg2_p
Hg2_c
ps160c_c
dhap_e
ps160e_c
dhap_p
protein
pe161_p
pe160c_c
3hcpalm5eACP_c
pe161_c
pepglycan_NM_p
pe141b_c
pe141b_p
capminusbiomass
lipid_NM_p
esslipid_NM_p
rna_c
pe160e_p
pe160e_c
pe160a_p
pe160a_c
pe160c_p
ps160b_c
biomass
dna_c
ps160a_c


In [32]:
#Correct missing formulas that did not import in Notebook 1D
model.metabolites.get_by_id("ps141b_c").formula ="C34H63N1O10P1"
model.metabolites.get_by_id("ps161_c").formula ="C38H68N1O10P1"
model.metabolites.get_by_id("pe160b_c").formula ="C35H70N1O8P1"
model.metabolites.get_by_id("pe160b_p").formula ="C35H70N1O8P1"
model.metabolites.get_by_id("ps160_c").formula ="C38H73N1O10P1"
model.metabolites.get_by_id("ps160e_c").formula ="C38H69N1O10P1"
model.metabolites.get_by_id("pe161_c").formula ="C37H69N1O8P1"
model.metabolites.get_by_id("pe161_p").formula ="C37H69N1O8P1"
model.metabolites.get_by_id("pe160c_c").formula ="C37H72N1O8P1"
model.metabolites.get_by_id("pe160c_p").formula ="C37H72N1O8P1"
model.metabolites.get_by_id("pe141_c").formula ="C33H62N1O8P1"
model.metabolites.get_by_id("pe141_p").formula ="C33H62N1O8P1"
model.metabolites.get_by_id("pe141b_c").formula ="C33H64N1O8P1"
model.metabolites.get_by_id("pe141b_p").formula ="C33H64N1O8P1"
model.metabolites.get_by_id("pe160a_c").formula ="C33H66N1O8P1"
model.metabolites.get_by_id("pe160a_p").formula ="C33H66N1O8P1"
model.metabolites.get_by_id("pe160b_c").formula ="C35H70N1O8P1"
model.metabolites.get_by_id("pe160b_p").formula ="C35H70N1O8P1"
model.metabolites.get_by_id("pe160c_c").formula ="C37H72N1O8P1"
model.metabolites.get_by_id("pe160c_p").formula ="C37H72N1O8P1"
model.metabolites.get_by_id("pe160e_c").formula ="C37H70N1O8P1"
model.metabolites.get_by_id("pe160e_p").formula ="C37H70N1O8P1"
model.metabolites.get_by_id("ps160a_c").formula ="C34H65N1O10P1"
model.metabolites.get_by_id("ps160b_c").formula ="C36H69N1O10P1"
model.metabolites.get_by_id("ps160c_c").formula ="C38H71N1O10P1"
model.metabolites.get_by_id("Hg2_p").formula ="Hg"
model.metabolites.get_by_id("Hg2_c").formula ="Hg"
model.metabolites.get_by_id("dhap_p").formula ="C3H5O6P"
model.metabolites.get_by_id("dhap_e").formula ="C3H5O6P"
model.metabolites.get_by_id("3hcpalm5eACP_c").formula ="C27H49N2O9PRS"



In [33]:
#Remaining metabolites with no formula are okay
no_formula = ("")
for metabolites in model.metabolites:
        if metabolites.formula == no_formula:
            print (metabolites.id)

essbiomass
protein
pepglycan_NM_p
capminusbiomass
lipid_NM_p
esslipid_NM_p
rna_c
biomass
dna_c


In [34]:
#Define biomass reactions to exclude from imbalance check
biomass_reactions = set(['PLIPIDS', 'PLIPIDS_ESS', 'PROTS', 'PEPGLY','RNAS', 'DNAS' ])


In [36]:
#Define remaining imbalances
#Many of these are due to the presence of cofactors in the reaction equation. In favor of correct stoichiometries, these cofactors are removed. 
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

142 142 142


,Reactions,Charge,Masses
0,3HAD161x,,"{'C': 29.0, 'H': 47.0, 'N': 2.0, 'O': 8.0, 'P'..."
1,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACCOAC,,"{'C': -9.999999999621423e-06, 'H': -1.49999999..."
3,ACHBS,,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
4,ACLS,,"{'C': -1.2e-06, 'H': -1.6e-06, 'O': -7e-07, 'N..."
...,...,...,...
137,THRD_L,,"{'C': -8.000000000230045e-07, 'H': -8.00000000..."
138,THRS,,"{'H': -8.000000004670937e-07, 'O': -5.99999999..."
139,TRDR1,,{'X': 1.0}
140,TSULST,,{'H': -2.0}


In [37]:
#Define cofactors
cofactors = set(['btn_c', 'thmpp_c', 'fad_c', 'pydx5p_c', 'nad_c', 'zn2_c', 'ca2_c'])

for metabolites in model.metabolites:
    if metabolites.id in cofactors:
        print (metabolites.id)



btn_c
thmpp_c
fad_c
pydx5p_c
nad_c
zn2_c
ca2_c


In [38]:
#Remove biotin as a cofactor
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "btn_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  btn_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('btn_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('btn_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.btn_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.btn_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

   ACCOAC :  btn_c : -1e-06
     BTS4 :  btn_c : 1.0


In [39]:
#Biotin has been removed as a cofactor all reactions
#example
model.reactions.ACCOAC

Reaction identifier,ACCOAC
Name,R_ACCOAC
Memory address,0x02043afe56a0
Stoichiometry,accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c Acetyl-CoA + ATP + Bicarbonate --> ADP + H + Malonyl-CoA + Phosphate
GPR,(NGO0821 or NGO0821) and NGO0044 and NGO0249 and NGO0045 and NGO2001
Lower bound,0.0
Upper bound,999999.0


In [40]:
#But Biotin has not been removed if it is an actual reactant
#example
model.reactions.BTS4

Reaction identifier,BTS4
Name,R_BTS4
Memory address,0x02043b042eb0
Stoichiometry,amet_c + dtbt_c + h_c --> btn_c + dad_5_c + h2o_c + met_L_c + o2_c S-Adenosyl-L-methionine + Dethiobiotin + H --> Biotin + 5-Deoxyadenosine + H2O + L-Methionine + O2
GPR,NGO0813 or NGO0813
Lower bound,0.0
Upper bound,999999.0


In [41]:
model.slim_optimize()

4.216064698818233

In [42]:
#Remove thmpp_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "thmpp_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  thmpp_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('thmpp_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('thmpp_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.thmpp_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.thmpp_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    ACHBS :  thmpp_c : -1e-07
     ACLS :  thmpp_c : -1e-07
       CS :  thmpp_c : -1e-06
     DXPS :  thmpp_c : -1e-07
      PDH :  thmpp_c : -1e-06


In [43]:
#example
model.reactions.ACHBS

Reaction identifier,ACHBS
Name,R_ACHBS
Memory address,0x02043afea7c0
Stoichiometry,2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c 2-Oxobutanoate + H + Pyruvate --> S-2-Aceto-2-hydroxybutanoate + CO2
GPR,NGO1236 and (Blank or NGO1235)
Lower bound,0.0
Upper bound,999999.0


In [44]:
#Fix naming for thiamine
#model.metabolites.get_by_id("the_c").id= "thm_e"
#model.metabolites.get_by_id("thp_c").id = "thm_p"
#model.metabolites.get_by_id("thc_c").id = "thm_c"

model.slim_optimize()

4.216072346346126

In [45]:
#Remove fad_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "fad_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  fad_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('fad_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('fad_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.fad_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.fad_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    ASPO3 :  fad_c : -1e-07
    ASPO5 :  fad_c : -1e-07
    ASPO6 :  fad_c : -1e-07
    CHORS :  fad_c : -1e-07
   DHORD2 :  fad_c : -1e-07
     FUMR :  fad_c : -1e-08
   MTHFR2 :  fad_c : -1e-07
 NADH16pp :  fad_c : -1e-07
    NADH5 :  fad_c : -1e-07
  NADHQNa :  fad_c : -1e-07
 NADPHQR2 :  fad_c : -1e-07
    PPPGO :  fad_c : -1e-07
    SUCD1 :  fad_c : -1e-08
    UAPGR :  fad_c : -1e-07


In [46]:
model.slim_optimize()

4.216072458380771

In [47]:
#Remove pydx5p_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "pydx5p_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  pydx5p_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('pydx5p_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('pydx5p_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.pydx5p_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.pydx5p_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

     ADCL :  pydx5p_c : -1e-07
     ADCS :  pydx5p_c : -1e-07
   AMAOTr :  pydx5p_c : -1e-07
    ARGDC :  pydx5p_c : -1e-07
     CYSS :  pydx5p_c : -1e-07
    DAPDC :  pydx5p_c : -1e-07
     PGCD :  pydx5p_c : -1e-07
   PSD120 :  pydx5p_c : -1e-07
   PSD140 :  pydx5p_c : -1e-07
   PSD141 :  pydx5p_c : -1e-07
  PSD141b :  pydx5p_c : -1e-07
   PSD160 :  pydx5p_c : -1e-07
  PSD160a :  pydx5p_c : -1e-07
  PSD160b :  pydx5p_c : -1e-07
  PSD160c :  pydx5p_c : -1e-07
  PSD160e :  pydx5p_c : -1e-07
  PSD161a :  pydx5p_c : -1e-07
   PSD180 :  pydx5p_c : -1e-07
   PSD181 :  pydx5p_c : -1e-07
    PSERT :  pydx5p_c : -1e-07
   SERD_L :  pydx5p_c : -1e-07
    SHSL5 :  pydx5p_c : -1e-07
   THRD_L :  pydx5p_c : -1e-07
     THRS :  pydx5p_c : -1e-07


In [48]:
model.slim_optimize()

4.216072785855977

In [49]:
#Remove nad_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "nad_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  nad_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('nad_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('nad_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.nad_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.nad_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

    AGMHE :  nad_c : -1e-07
   AMPMS2 :  nad_c : -1.0
     DHQS :  nad_c : -1e-07
  EAR161a :  nad_c : 1.0
 GLYCLTDx :  nad_c : 1.0
     IPPS :  nad_c : -1e-07
  NADHQNa :  nad_c : 1.0


In [50]:
#Remove zn2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "zn2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  zn2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('zn2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('zn2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.zn2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.zn2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

AMPTASECG :  zn2_c : -1e-07
     METS :  zn2_c : -1e-07
   PPBNGS :  zn2_c : -1e-07


In [51]:
#Remove ca2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "ca2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  ca2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('ca2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('ca2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.ca2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.ca2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

PLIPA1A120pp :  ca2_c : -1e-07
PLIPA1A140pp :  ca2_c : -1e-07
PLIPA1A141b :  ca2_c : -1e-07
PLIPA1A141pp :  ca2_c : -1e-07
PLIPA1A160a :  ca2_c : -1e-07
PLIPA1A160b :  ca2_c : -1e-07
PLIPA1A160c :  ca2_c : -1e-07
PLIPA1A160e :  ca2_c : -1e-07
PLIPA1A160pp :  ca2_c : -1e-07
PLIPA1A161a :  ca2_c : -1e-07
PLIPA1A180pp :  ca2_c : -1e-07
PLIPA1A181pp :  ca2_c : -1e-07
PLIPA1E120pp :  ca2_c : -1e-07
PLIPA1E140pp :  ca2_c : -1e-07
PLIPA1E141b :  ca2_c : -1e-07
PLIPA1E141pp :  ca2_c : -1e-07
PLIPA1E160a :  ca2_c : -1e-07
PLIPA1E160b :  ca2_c : -1e-07
PLIPA1E160c :  ca2_c : -1e-07
PLIPA1E160e :  ca2_c : -1e-07
PLIPA1E160pp :  ca2_c : -1e-07
PLIPA1E161a :  ca2_c : -1e-07
PLIPA1E180pp :  ca2_c : -1e-07
PLIPA1E181pp :  ca2_c : -1e-07
PLIPA1G120pp :  ca2_c : -1e-07
PLIPA1G140pp :  ca2_c : -1e-07
PLIPA1G141b :  ca2_c : -1e-07
PLIPA1G141pp :  ca2_c : -1e-07
PLIPA1G160a :  ca2_c : -1e-07
PLIPA1G160b :  ca2_c : -1e-07
PLIPA1G160c :  ca2_c : -1e-07
PLIPA1G160e :  ca2_c : -1e-07
PLIPA1G160pp :  ca2_c : -

In [52]:
#Remove mn2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "mn2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  mn2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('mn2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('mn2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.mn2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.mn2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

  SPODMMn :  mn2_c : -1e-07


In [53]:
#Remove fe2_c
cofactor_rxns =[]
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in react.id:
        continue
    if "DM_" in react.id:
        continue
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        if "fe2_c" in react.reaction:
            cofactor_rxns.append(react.id)
            print('%9s :  fe2_c : %s' % (react.id, react._metabolites[model.metabolites.get_by_id('fe2_c')]))
            original_coefficient = react._metabolites[model.metabolites.get_by_id('fe2_c')]
            if abs(original_coefficient) >100:
                react.subtract_metabolites({model.metabolites.fe2_c: original_coefficient})
            if abs(original_coefficient) <0.001:
                react.subtract_metabolites({model.metabolites.fe2_c: original_coefficient})
            
            
if len(cofactor_rxns) == 0:
        print ('No reactions')

PHEMEFErel :  fe2_c : 1.0
    PROHX :  fe2_c : -1e-07


In [54]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

70 70 70


,Reactions,Charge,Masses
0,3HAD161x,,"{'C': 29.0, 'H': 47.0, 'N': 2.0, 'O': 8.0, 'P'..."
1,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACPH,,"{'H': -2.0, 'O': -1.0, 'R': -1.0}"
3,ACPS1,,"{'H': 1.0, 'O': 1.0, 'R': 1.0}"
4,AGPAT160e,,"{'C': 2.0, 'H': 4.0}"
...,...,...,...
65,SHSL5,,{'H': -1.0}
66,SIALT1,,"{'C': -11.0, 'H': -18.0, 'N': -1.0, 'O': -8.0}"
67,TFFE,,{'Fe': 1.0}
68,TRDR1,,{'X': 1.0}


In [55]:
model.slim_optimize()

4.216072785932971

In [56]:
#Correct Imbalances for ACP reactions
#ACPH
r = model.reactions.get_by_id('ACPH')
print(r.check_mass_balance())
r


{'H': -2.0, 'O': -1.0, 'R': -1.0}


Reaction identifier,ACPH
Name,R_ACPH
Memory address,0x02043afee760
Stoichiometry,ACP_c + h2o_c --> pan4p_c acyl-carrier-protein + H2O --> Pantetheine-4-phosphate
GPR,Orphan
Lower bound,0.0
Upper bound,999999.0


In [57]:
apoACP_c = Metabolite(
    'apoACP_c',
    formula='HOR',
    name='apo acyl-carrier-protein',
    compartment='cytosol')

h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    apoACP_c: 1.0,
    h_c: 1.0})

print(r)
print(r.check_mass_balance())

ACPH: ACP_c + h2o_c --> apoACP_c + h_c + pan4p_c
{}


In [58]:
model.slim_optimize()

4.216072785932986

In [59]:
#ACPS1
r = model.reactions.get_by_id('ACPS1')
print(r.check_mass_balance())
r


{'H': 1.0, 'O': 1.0, 'R': 1.0}


Reaction identifier,ACPS1
Name,R_ACPS1
Memory address,0x02043aff26a0
Stoichiometry,coa_c --> ACP_c + h_c + pap_c Coenzyme-A --> acyl-carrier-protein + H + Adenosine-3-5-bisphosphate
GPR,NGO1507
Lower bound,0.0
Upper bound,999999.0


In [60]:
r.add_metabolites({
    apoACP_c: -1.0})

print(r)
print(r.check_mass_balance())

ACPS1: apoACP_c + coa_c --> ACP_c + h_c + pap_c
{}


In [61]:
#Add apoACP import reaction. This is required to show "synthesis" of a protein based cofactor. 
#Integration of Biomass Formulations of Genome-Scale Metabolic Models with Experimental Data Reveals Universally Essential Cofactors in Prokaryotes
#Joana C Xavier, Kiran Raosaheb Patil, Isabel Rocha
#PMID: 27939572 PMCID: PMC5249239

apoACP_c = model.metabolites.get_by_id('apoACP_c')

EX_apoACP_c_ = cobra.Reaction('EX_apoACP_c_')
EX_apoACP_c_.notes = {'citations': 'PMCID: PMC5249239', 'type': 'essential cofactor', 'annotation': ''}
EX_apoACP_c_.name = 'exchange of apoACP'
EX_apoACP_c_.gene_reaction_rule = ''
EX_apoACP_c_.lower_bound = 0.
EX_apoACP_c_.upper_bound = 1000.
EX_apoACP_c_.add_metabolites({
    apoACP_c: -1.0,
  
})

model.add_reactions([EX_apoACP_c_])
new_rxns += 1

In [62]:
r=model.reactions.get_by_id('EX_apoACP_c_')
r

Reaction identifier,EX_apoACP_c_
Name,exchange of apoACP
Memory address,0x02043afee3d0
Stoichiometry,apoACP_c --> apo acyl-carrier-protein -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [65]:
complete(model)
model.slim_optimize()

4.216072785932984

In [66]:
#AMPMS2
r = model.reactions.get_by_id('AMPMS2')
print(r.check_mass_balance())
r

{'H': 1.0, 'O': -3.0, 'P': -1.0}


Reaction identifier,AMPMS2
Name,R_AMPMS2
Memory address,0x02043b0227f0
Stoichiometry,air_c + h2o_c + nad_c --> 4ahmmp_c + 2.0 for_c + 3.0 h_c + nadh_c 5-amino-1-5-phospho-D-ribosyl-imidazole + H2O + Nicotinamide-adenine-dinucleotide --> 4-Amino-5-hydroxymethyl-2-methylpyrimidine + 2.0 Formate + 3.0 H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO2041
Lower bound,0.0
Upper bound,999999.0


In [67]:
#The reaction AMPMS2 should use 4ampm rather than 4ahmmp according to BIGG and KEGG. Replaced this metabolite in the reaction.


In [68]:
met1=model.metabolites.get_by_id('4ampm_c')
met2 = model.metabolites.get_by_id('4ahmmp_c')

r.subtract_metabolites({
    met2: 1.0})
r.add_metabolites({
    met1: 1.0})

print(r.check_mass_balance())
r

{}


Reaction identifier,AMPMS2
Name,R_AMPMS2
Memory address,0x02043b0227f0
Stoichiometry,air_c + h2o_c + nad_c --> 4ampm_c + 2.0 for_c + 3.0 h_c + nadh_c 5-amino-1-5-phospho-D-ribosyl-imidazole + H2O + Nicotinamide-adenine-dinucleotide --> 4-Amino-2-methyl-5-phosphomethylpyrimidine + 2.0 Formate + 3.0 H + Nicotinamide-adenine-dinucleotide--reduced
GPR,NGO2041
Lower bound,0.0
Upper bound,999999.0


In [69]:
#However, now model doesn't grow in RPMI which doesn't contain added thiamine. Thiamine synthesis seems to have been disrupted by changing the metabolite. 
model.slim_optimize()

4.216072785932965

In [70]:
#ASNSYN
r = model.reactions.get_by_id('ASNSYN')
print(r.check_mass_balance())
r

{'H': 2.0, 'O': 2.0}


Reaction identifier,ASNSYN
Name,R_ASNSYN
Memory address,0x02043b030fd0
Stoichiometry,asp_L_c + atp_c + gln_L_c + h_c --> adp_c + asn_L_c + glu_L_c + h2o_c + pi_c L-Aspartate + ATP + L-Glutamine + H --> ADP + L-Asparagine + L-Glutamate + H2O + Phosphate
GPR,NGO1489 and NGO0663 and NGO0662 and NGO0660
Lower bound,0.0
Upper bound,999999.0


In [71]:
#BTS4
r = model.reactions.get_by_id('BTS4')
print(r.check_mass_balance())
r

{'O': 3.0, 'S': 1.0}


Reaction identifier,BTS4
Name,R_BTS4
Memory address,0x02043b042eb0
Stoichiometry,amet_c + dtbt_c + h_c --> btn_c + dad_5_c + h2o_c + met_L_c + o2_c S-Adenosyl-L-methionine + Dethiobiotin + H --> Biotin + 5-Deoxyadenosine + H2O + L-Methionine + O2
GPR,NGO0813 or NGO0813
Lower bound,0.0
Upper bound,999999.0


In [72]:
#Reaction will remain imbalanced for Sulfer. The sulfer is donated by the enzyme itself and not by a metabolite.
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')
o2_c = model.metabolites.get_by_id('o2_c')

r.add_metabolites({
    h2o_c: -1.0,
    h_c: 2.0,
    o2_c: -1.0
})

print(r)
print(r.check_mass_balance())

BTS4: amet_c + dtbt_c --> btn_c + dad_5_c + h_c + met_L_c
{'S': 1.0}


In [73]:
#CTPS3
r = model.reactions.get_by_id('CTPS3')
print(r.check_mass_balance())
r

{'H': 1.0}


Reaction identifier,CTPS3
Name,R_CTPS3
Memory address,0x02043b05c970
Stoichiometry,ctp_c + h2o_c <=> nh4_c + utp_c CTP + H2O <=> Ammonium + UTP
GPR,NGO0421
Lower bound,-999999.0
Upper bound,999999.0


In [74]:
nh3_c= Metabolite(
    'nh3_c',
    formula='NH3',
    name='Ammonia',
    compartment='cytosol')

nh4_c = model.metabolites.get_by_id('nh4_c')


r.add_metabolites({
    nh4_c: -1.0,
    nh3_c: 1.0,
})

print(r)
print(r.check_mass_balance())

CTPS3: ctp_c + h2o_c <=> nh3_c + utp_c
{}


In [75]:
model.slim_optimize()

4.208458953428455

In [76]:
#DHORD2
r = model.reactions.get_by_id('DHORD2')
print(r.check_mass_balance())
r

{'C': -1.6999999985500835e-06, 'H': -1.900000000887303e-06, 'N': -3.9999999978945766e-07, 'O': -8.999999998593466e-07, 'P': -1e-07}


Reaction identifier,DHORD2
Name,R_DHORD2
Memory address,0x02043b081d00
Stoichiometry,dhor_S_c + 1e-07 fmn_c + q8_c --> orot_c + q8h2_c S-Dihydroorotate + 1e-07 FMN + Ubiquinone-8 --> Orotate + Ubiquinol-8
GPR,NGO1761
Lower bound,0.0
Upper bound,999999.0


In [77]:
fmn_c = model.metabolites.get_by_id('fmn_c')



r.add_metabolites({
    fmn_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

DHORD2: dhor_S_c + q8_c --> orot_c + q8h2_c
{}


In [78]:
#DHQS
r = model.reactions.get_by_id('DHQS')
print(r.check_mass_balance())
r

{'Co': -1e-07}


Reaction identifier,DHQS
Name,R_DHQS
Memory address,0x02043b089550
Stoichiometry,2dda7p_c + 1e-07 cobalt2_c --> 3dhq_c + pi_c 2-Dehydro-3-deoxy-D-arabino-heptonate-7-phosphate + 1e-07 Co2 --> 3-Dehydroquinate + Phosphate
GPR,NGO0092
Lower bound,0.0
Upper bound,999999.0


In [79]:
cobalt2_c = model.metabolites.get_by_id('cobalt2_c')



r.add_metabolites({
    cobalt2_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

DHQS: 2dda7p_c --> 3dhq_c + pi_c
{}


In [80]:
model.slim_optimize()

4.208458953428458

In [81]:
#GLYCLTDx
r = model.reactions.get_by_id('GLYCLTDx')
print(r.check_mass_balance())
r

{'H': 1.0}


Reaction identifier,GLYCLTDx
Name,R_GLYCLTDx
Memory address,0x02043b17d6d0
Stoichiometry,glx_c + nadh_c --> glyclt_c + nad_c Glyoxylate + Nicotinamide-adenine-dinucleotide--reduced --> Glycolate + Nicotinamide-adenine-dinucleotide
GPR,NGO2043
Lower bound,0.0
Upper bound,999999.0


In [82]:
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1,
})

print(r)
print(r.check_mass_balance())

GLYCLTDx: glx_c + h_c + nadh_c --> glyclt_c + nad_c
{}


In [83]:
#GLYCLTDy
r = model.reactions.get_by_id('GLYCLTDy')
print(r.check_mass_balance())
r

{'H': 1.0}


Reaction identifier,GLYCLTDy
Name,R_GLYCLTDy
Memory address,0x02043b17d880
Stoichiometry,glx_c + nadph_c --> glyclt_c + nadp_c Glyoxylate + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> Glycolate + Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO2043
Lower bound,0.0
Upper bound,999999.0


In [84]:
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1,
})

print(r)
print(r.check_mass_balance())

GLYCLTDy: glx_c + h_c + nadph_c --> glyclt_c + nadp_c
{}


In [85]:
#H2AKGAL
r = model.reactions.get_by_id('H2AKGAL')
print(r.check_mass_balance())
r

{'H': -1.0}


Reaction identifier,H2AKGAL
Name,R_H2AKGAL
Memory address,0x02043b196e50
Stoichiometry,4h2oglt_c --> glx_c + h_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + H + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,999999.0


In [86]:
#H2AKGAL is not consistent with the bigg name for this same reaction. Corrected name to DDPGA. 
#Similarly, the protenation of 4h2oglt_c was used from kegg, which is incorrect for neutral ph.
#corrected to the formula from bigg. And removed the now unnecessary hydrogens from the reaction.
r.id  ='DDPGA'
r.name ='R_DDPGA'
model.metabolites.get_by_id('4h2oglt_c').formula = 'C5H4O6'
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: 1.0,
})

print(r)
print(r.check_mass_balance())

DDPGA: 4h2oglt_c --> glx_c + 2.0 h_c + pyr_c
{'H': 2.0}


In [87]:
#HGTA
r = model.reactions.get_by_id('HGTA')
print(r.check_mass_balance())
r

{}


Reaction identifier,HGTA
Name,R_HGTA
Memory address,0x02043b1a3dc0
Stoichiometry,akg_c + e4hglu_c <=> 4h2oglt_c + glu_L_c + h_c 2-Oxoglutarate + L-erythro-4-Hydroxyglutamate <=> D-4-Hydroxy-2-oxoglutarate + L-Glutamate + H
GPR,NGO1452
Lower bound,-999999.0
Upper bound,999999.0


In [88]:
#HGTA is not consistent with the bigg name for this same reaction. Corrected name to EHGLAT. 
#Similarly, the protenation of e4hglu_c was used from kegg, which is incorrect for neutral ph.
#corrected to the formula from bigg. And removed the now unnecessary hydrogens from the reaction.
r.id  ='EHGLAT'
r.name ='R_EHGLAT'
model.metabolites.get_by_id('e4hglu_c').formula = 'C5H8NO5'
h_c = model.metabolites.get_by_id('h_c')



r.add_metabolites({
    h_c: -1.0,
})

print(r)
print(r.check_mass_balance())

EHGLAT: akg_c + e4hglu_c <=> 4h2oglt_c + glu_L_c
{}


In [90]:
model.slim_optimize()

4.208458953428458

In [91]:
#HMT
r = model.reactions.get_by_id('HMT')
print(r.check_mass_balance())
r

{'H': -1.0}


Reaction identifier,HMT
Name,R_HMT
Memory address,0x02043b1b0e20
Stoichiometry,amet_c + his_L_c --> ahcys_c + mhis_c S-Adenosyl-L-methionine + L-Histidine --> S-Adenosyl-L-homocysteine + N(pi)-Methyl-L-histidine
GPR,NGO0746
Lower bound,0.0
Upper bound,999999.0


In [92]:
model.remove_reactions('r')

C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")
C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:738: UserWarning: r not in AE004969.1
  warn("%s not in %s" % (reaction, self))


In [93]:
#MCITS
r = model.reactions.get_by_id('MCITS')
print(r.check_mass_balance())
r

{'H': -3.0}


Reaction identifier,MCITS
Name,R_MCITS
Memory address,0x02043b206fa0
Stoichiometry,h2o_c + 2.0 h_c + oaa_c + ppcoa_c --> 2mcit_c + coa_c H2O + 2.0 H + Oxaloacetate + Propanoyl-CoA --> 2-Methylcitrate + Coenzyme-A
GPR,NGO1525
Lower bound,0.0
Upper bound,999999.0


In [94]:
h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 3.0,
})

print(r)
print(r.check_mass_balance())

MCITS: h2o_c + oaa_c + ppcoa_c --> 2mcit_c + coa_c + h_c
{}


In [95]:
#MTHF
r = model.reactions.get_by_id('MTHF')
print(r.check_mass_balance())
r

{'H': -3.0, 'O': 1.0}


Reaction identifier,MTHF
Name,R_MTHF
Memory address,0x02043b239eb0
Stoichiometry,5mthf_c --> 10fthf_c 5-Methyltetrahydrofolate --> 10-Formyltetrahydrofolate
GPR,Sgene
Lower bound,0.0
Upper bound,999999.0


In [96]:
#MTHF is a spontaneous reaction. Uncertain how frequently this occurs in nature. 
#Corrected h2o and hydrogens.
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')


r.add_metabolites({
    h_c: 5.0,
    h2o_c: -1.0
})

print(r)
print(r.check_mass_balance())

MTHF: 5mthf_c + h2o_c --> 10fthf_c + 5.0 h_c
{}


In [97]:
#NADHQNa
r = model.reactions.get_by_id('NADHQNa')
print(r.check_mass_balance())
r

{'C': -1.6999999985500835e-06, 'H': -1.900000000887303e-06, 'N': -4.0000000023354687e-07, 'O': -8.999999998593466e-07, 'P': -1.0000000005838672e-07}


Reaction identifier,NADHQNa
Name,R_NADHQNa
Memory address,0x02043b245d90
Stoichiometry,1e-07 fmn_c + h_c + 2.0 na1_c + nadh_c + q8_c --> 2.0 na1_p + nad_c + q8h2_c 1e-07 FMN + H + 2.0 Sodium + Nicotinamide-adenine-dinucleotide--reduced + Ubiquinone-8 --> 2.0 Sodium + Nicotinamide-adenine-dinucleotide + Ubiquinol-8
GPR,NGO1418 and NGO1417 and NGO1416 and NGO1415 and NGO1414 and NGO1413
Lower bound,0.0
Upper bound,999999.0


In [98]:
fmn_c = model.metabolites.get_by_id('fmn_c')


r.add_metabolites({
    fmn_c: 1e-07,
})

print(r)
print(r.check_mass_balance())

NADHQNa: h_c + 2.0 na1_c + nadh_c + q8_c --> 2.0 na1_p + nad_c + q8h2_c
{}


In [100]:
#Imbalance progress
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

59 59 59


,Reactions,Charge,Masses
0,3HAD161x,,"{'C': 29.0, 'H': 47.0, 'N': 2.0, 'O': 8.0, 'P'..."
1,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,AGPAT160e,,"{'C': 2.0, 'H': 4.0}"
3,ASNSYN,,"{'H': 2.0, 'O': 2.0}"
4,BTS4,,{'S': 1.0}
5,CYTCS,,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
6,DASYN141b,,"{'C': -4.0, 'H': -8.0}"
7,DASYN160e,,"{'C': -4.0, 'H': -12.0}"
8,DASYN161a,,{'H': -1.0}
9,EAR161a,,"{'C': -29.0, 'H': -47.0, 'N': -2.0, 'O': -8.0,..."


In [101]:
model.slim_optimize()

4.208458953428469

In [102]:
r= model.reactions.get_by_id('PLIPIDS')
print(r)

PLIPIDS: 0.001857 pa120_p + 0.001157 pa140_p + 0.001202 pa141_p + 0.003838 pa160_p + 0.006675 pa161_p + 0.000484 pa180_p + 0.0008971 pa181_p + 0.004466 pe120_p + 0.010539 pe140_p + 0.002913 pe141_p + 0.004721 pe141b_p + 0.009371 pe160_p + 0.01007 pe160a_p + 0.02591 pe160c_p + 0.0186 pe160e_p + 0.01562 pe161_p + 0.001189 pe180_p + 0.022 pe181_p + 0.001551 pg120_p + 0.001283 pg140_p + 0.001016 pg141_p + 0.001668 pg141b_p + 0.003285 pg160_p + 0.002712 pg160a_p + 0.001644 pg160b_p + 0.006129 pg160c_p + 0.006099 pg161_p + 0.000418 pg180_p + 0.000774 pg181_p <=> 0.1483 lipid_NM_p


In [103]:
#Phospholipids adjusted for Gc specific values based on "Biomass composition from nmb_itm560 updated for Gc" spreadsheet
r= model.reactions.get_by_id('PLIPIDS')

pe141b_p= model.metabolites.get_by_id('pe141b_p')
pe160a_p= model.metabolites.get_by_id('pe160a_p')
pe160c_p= model.metabolites.get_by_id('pe160c_p')
pe160e_p= model.metabolites.get_by_id('pe160e_p')
pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pa120_p= model.metabolites.get_by_id('pa120_p')
pa140_p= model.metabolites.get_by_id('pa140_p')
pa141_p= model.metabolites.get_by_id('pa141_p')
pa160_p= model.metabolites.get_by_id('pa160_p')
pa161_p= model.metabolites.get_by_id('pa161_p')
pa180_p= model.metabolites.get_by_id('pa180_p')
pa181_p= model.metabolites.get_by_id('pa181_p')

pe120_p= model.metabolites.get_by_id('pe120_p')
pe140_p= model.metabolites.get_by_id('pe140_p')
pe141_p= model.metabolites.get_by_id('pe141_p')
pe160_p= model.metabolites.get_by_id('pe160_p')
pe161_p= model.metabolites.get_by_id('pe161_p')
pe180_p= model.metabolites.get_by_id('pe180_p')
pe181_p= model.metabolites.get_by_id('pe181_p')

pg120_p= model.metabolites.get_by_id('pg120_p')
pg140_p= model.metabolites.get_by_id('pg140_p')
pg141_p= model.metabolites.get_by_id('pg141_p')
pg160_p= model.metabolites.get_by_id('pg160_p')
pg161_p= model.metabolites.get_by_id('pg161_p')
pg180_p= model.metabolites.get_by_id('pg180_p')
pg181_p= model.metabolites.get_by_id('pg181_p')


r.add_metabolites({
 
    pa120_p:  0.001857, 
    pa140_p: 0.001157 ,
    pa160_p: 0.003838,
    pa180_p:  0.000484,
    pa141_p: 0.001202 ,
    pa161_p: 0.006675,
    pa181_p: 0.0008971, 
    pe120_p: 0.004466 ,
    pe140_p: 0.010539 ,
    pe141b_p: 0.004721, 
    pe160_p: 0.009371 ,
    pe160a_p: 0.01007 ,
    pe160c_p: 0.02591 ,
    pe160e_p: 0.0186,
    pe180_p: 0.001189, 
    pe141_p: 0.002913 ,
    pe161_p: 0.01562 ,
    pe181_p:  0.022 ,
    pg120_p: 0.001551 ,
    pg140_p: 0.001283 ,
    pg160_p: 0.003285 ,
    pg160a_p:  0.002712, 
    pg160b_p: 0.001644 ,
    pg160c_p:  0.006129, 
    pg161_p: 0.006099 ,
    pg181_p: 0.000774 ,
    pg180_p: 0.000418 ,
    pg141_p: 0.001016 ,
    pg141b_p: 0.001668, })

  
r.add_metabolites({   
    pa120_p: -0.000503555042155509,
    pa160_p: -0.00207597350252432 ,
    pa180_p: -0.000290080640591335,
    pa141_p: -0.000371731800974898 ,
    pa161_p: -0.00564538392418283 ,
    pa181_p: -0.00216477129604764 ,
    pe120_p: -0.00234345993674653 ,
    pe140_p: -0.0025598346977071 ,
    pe160_p: -0.055748823259178,
    pe180_p: -0.00137834776769998,
    pe141_p: -0.00821977537556325,
    pe161_p: -0.0255682154771119,
    pe181_p: -0.0102822273478348 ,
    pg120_p: -0.000703620745734723,
    pg140_p: -3.03207570481577E-05,
    pg160_p: -0.0122303684865845,
    pg180_p: -0.000418944804169265,
    pg141_p: -0.00290595573235281,
    pg161_p: -0.00862596402374158,
    pg181_p: -0.00312454317572049,
})

print(r)

PLIPIDS: 0.000503555042155509 pa120_p + 0.000371731800974898 pa141_p + 0.00207597350252432 pa160_p + 0.00564538392418283 pa161_p + 0.000290080640591335 pa180_p + 0.00216477129604764 pa181_p + 0.00234345993674653 pe120_p + 0.0025598346977071 pe140_p + 0.00821977537556325 pe141_p + 0.055748823259178 pe160_p + 0.0255682154771119 pe161_p + 0.00137834776769998 pe180_p + 0.0102822273478348 pe181_p + 0.000703620745734723 pg120_p + 3.03207570481577e-05 pg140_p + 0.00290595573235281 pg141_p + 0.0122303684865845 pg160_p + 0.00862596402374158 pg161_p + 0.000418944804169265 pg180_p + 0.00312454317572049 pg181_p <=> 0.1483 lipid_NM_p


In [104]:
model.slim_optimize()

4.208458953428481

In [105]:
r=model.reactions.get_by_id('PLIPIDS_ESS')
print(r)

PLIPIDS_ESS: 0.001857 pa120_p + 0.001157 pa140_p + 0.001202 pa141_p + 0.003838 pa160_p + 0.006675 pa161_p + 0.000484 pa180_p + 0.0008971 pa181_p + 0.001551 pg120_p + 0.001283 pg140_p + 0.001016 pg141_p + 0.001668 pg141b_p + 0.003285 pg160_p + 0.002712 pg160a_p + 0.001644 pg160b_p + 0.006129 pg160c_p + 0.006099 pg161_p + 0.000418 pg180_p + 0.000774 pg181_p <=> 0.04269 esslipid_NM_p


In [106]:
r= model.reactions.get_by_id('PLIPIDS_ESS')

pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pe141b_p= model.metabolites.get_by_id('pe141b_p')
pe160a_p= model.metabolites.get_by_id('pe160a_p')
pe160c_p= model.metabolites.get_by_id('pe160c_p')
pe160e_p= model.metabolites.get_by_id('pe160e_p')
pg141b_p= model.metabolites.get_by_id('pg141b_p')
pg160a_p= model.metabolites.get_by_id('pg160a_p')
pg160b_p= model.metabolites.get_by_id('pg160b_p')
pg160c_p= model.metabolites.get_by_id('pg160c_p')

pa120_p= model.metabolites.get_by_id('pa120_p')
pa140_p= model.metabolites.get_by_id('pa140_p')
pa141_p= model.metabolites.get_by_id('pa141_p')
pa160_p= model.metabolites.get_by_id('pa160_p')
pa161_p= model.metabolites.get_by_id('pa161_p')
pa180_p= model.metabolites.get_by_id('pa180_p')
pa181_p= model.metabolites.get_by_id('pa181_p')

pe120_p= model.metabolites.get_by_id('pe120_p')
pe140_p= model.metabolites.get_by_id('pe140_p')
pe141_p= model.metabolites.get_by_id('pe141_p')
pe160_p= model.metabolites.get_by_id('pe160_p')
pe161_p= model.metabolites.get_by_id('pe161_p')
pe180_p= model.metabolites.get_by_id('pe180_p')
pe181_p= model.metabolites.get_by_id('pe181_p')

pg120_p= model.metabolites.get_by_id('pg120_p')
pg140_p= model.metabolites.get_by_id('pg140_p')
pg141_p= model.metabolites.get_by_id('pg141_p')
pg160_p= model.metabolites.get_by_id('pg160_p')
pg161_p= model.metabolites.get_by_id('pg161_p')
pg180_p= model.metabolites.get_by_id('pg180_p')
pg181_p= model.metabolites.get_by_id('pg181_p')


r.add_metabolites({
 
    pa120_p:  0.001857, 
    pa140_p: 0.001157 ,
    pa141_p: 0.001202 ,
    pa160_p: 0.003838,
    pa161_p: 0.006675,
    pa180_p:  0.000484,
    pa181_p: 0.0008971, 

    pg120_p: 0.001551 ,
    pg140_p: 0.001283 ,
    pg160_p: 0.003285 ,
    pg160a_p:  0.002712, 
    pg160b_p: 0.001644 ,
    pg160c_p:  0.006129, 
    pg161_p: 0.006099 ,
    pg181_p: 0.000774 ,
    pg180_p: 0.000418 ,
    pg141_p: 0.001016 ,
    pg141b_p: 0.001668, })

  
r.add_metabolites({ 
    pa120_p: -0.000503555042155509,
    
    pa141_p: -0.000371731800974898 ,
    pa160_p: -0.00207597350252432 ,
    pa161_p: -0.00564538392418283 ,
    pa180_p: -0.000290080640591335,
    pa181_p: -0.00216477129604764 ,
  
    pg120_p: -0.000703620745734723,
    pg140_p: -3.03207570481577E-05,
    pg141_p: -0.00290595573235281,
    pg160_p: -0.0122303684865845,
    pg161_p: -0.00862596402374158,
    pg180_p: -0.000418944804169265,
    pg181_p: -0.00312454317572049, })

print(r)

PLIPIDS_ESS: 0.000503555042155509 pa120_p + 0.000371731800974898 pa141_p + 0.00207597350252432 pa160_p + 0.00564538392418283 pa161_p + 0.000290080640591335 pa180_p + 0.00216477129604764 pa181_p + 0.000703620745734723 pg120_p + 3.03207570481577e-05 pg140_p + 0.00290595573235281 pg141_p + 0.0122303684865845 pg160_p + 0.00862596402374158 pg161_p + 0.000418944804169265 pg180_p + 0.00312454317572049 pg181_p <=> 0.04269 esslipid_NM_p


In [107]:
model.slim_optimize()

4.208806878287181

In [108]:
model.remove_reactions('AGPAT141b')
model.remove_reactions('AGPAT160b')
model.remove_reactions('AGPAT160c')
model.remove_reactions('AGPAT160e')




C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [100]:
model.slim_optimize()

0.26369251928851023

In [109]:
model.remove_reactions('DASYN160a')
model.remove_reactions('DASYN160b')
model.remove_reactions('DASYN160c')
model.remove_reactions('DASYN160e')
model.remove_reactions('DASYN141b')

In [110]:
model.slim_optimize()

4.208806878287171

In [111]:
model.remove_reactions('PGPP141b')
model.remove_reactions('PGPP160a')
model.remove_reactions('PGPP160b')
model.remove_reactions('PGPP160c')
model.remove_reactions('PGPP160e')

In [112]:
model.slim_optimize()

4.208806878287183

In [113]:
model.remove_reactions('PGPP160app')
model.remove_reactions('PGPP160bpp')
model.remove_reactions('PGPP160cpp')
model.remove_reactions('PGPP160epp')
model.remove_reactions('PGPP141bpp')

In [114]:
model.slim_optimize()

4.208806878287171

In [115]:
model.remove_reactions('PGSA160a')
model.remove_reactions('PGSA160b')
model.remove_reactions('PGSA160c')
model.remove_reactions('PGSA160e')
model.remove_reactions('PGSA141b')

In [116]:
model.slim_optimize()

4.208806878287193

In [117]:
model.remove_reactions('PLIPA1A160a')
model.remove_reactions('PLIPA1A160b')
model.remove_reactions('PLIPA1A160c')
model.remove_reactions('PLIPA1A160e')
model.remove_reactions('PLIPA1A141b')

In [110]:
model.slim_optimize()

0.2636925192880822

In [118]:
model.remove_reactions('PLIPA1E160a')
model.remove_reactions('PLIPA1E160b')
model.remove_reactions('PLIPA1E160c')
model.remove_reactions('PLIPA1E160e')
model.remove_reactions('PLIPA1E141b')

In [119]:
model.slim_optimize()

4.208806878287195

In [120]:
model.remove_reactions('PLIPA1G160a')
model.remove_reactions('PLIPA1G160b')
model.remove_reactions('PLIPA1G160c')
model.remove_reactions('PLIPA1G160e')
model.remove_reactions('PLIPA1G141b')

In [121]:
model.slim_optimize()

4.2088068782871835

In [122]:
model.remove_reactions('PSD160a')
model.remove_reactions('PSD160b')
model.remove_reactions('PSD160c')
model.remove_reactions('PSD160e')
model.remove_reactions('PSD141b')

In [116]:
model.slim_optimize()

0.26369251928858745

In [123]:
model.remove_reactions('PSSA160a')
model.remove_reactions('PSSA160b')
model.remove_reactions('PSSA160c')
model.remove_reactions('PSSA160e')
model.remove_reactions('PSSA141b')

In [124]:
model.slim_optimize()

4.208806878287189

In [125]:
model.remove_reactions('2AGPA181atipp')

In [126]:
model.slim_optimize()

4.208806878287169

In [127]:
model.remove_reactions('PA141babcpp')
model.remove_reactions('PA160aabcpp')
model.remove_reactions('PA160babcpp')
model.remove_reactions('PA160cabcpp')
model.remove_reactions('PA160eabcpp')
model.remove_reactions('PE160aabcpp')
model.remove_reactions('PE160babcpp')
model.remove_reactions('PE160cabcpp')
model.remove_reactions('PE160eabcpp')
model.remove_reactions('PG141babcpp')
model.remove_reactions('PG160aabcpp')
model.remove_reactions('PG160babcpp')
model.remove_reactions('PG160cabcpp')
model.remove_reactions('PG160eabcpp')

In [128]:
model.remove_reactions('PGP141babcpp')
model.remove_reactions('PGP160aabcpp')
model.remove_reactions('PGP160babcpp')
model.remove_reactions('PGP160cabcpp')
model.remove_reactions('PGP160eabcpp')

In [129]:
model.slim_optimize()

4.208806878287175

In [130]:
#GARFT2
r = model.reactions.get_by_id('GARFT2')
print(r.check_mass_balance())
r

{'H': -2.0}


Reaction identifier,GARFT2
Name,R_GARFT2
Memory address,0x02043b14dd30
Stoichiometry,gar_c + h2o_c + methf_c --> fgam_c + thf_c N1-5-Phospho-D-ribosyl-glycinamide + H2O + 5-10-Methenyltetrahydrofolate --> N2-Formyl-N1-5-phospho-D-ribosyl-glycinamide + 5-6-7-8-Tetrahydrofolate
GPR,NGO1224
Lower bound,0.0
Upper bound,999999.0


In [131]:
h_c = model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2.0,
})

print(r)
print(r.check_mass_balance())

GARFT2: gar_c + h2o_c + methf_c --> fgam_c + 2.0 h_c + thf_c
{}


In [132]:
model.slim_optimize()

4.208806878287196

In [133]:
#GLCTR1
r1 = model.reactions.get_by_id('GLCTR1')
print(r1.check_mass_balance())
r1

{'O': -5.0}


Reaction identifier,GLCTR1
Name,R_GLCTR1
Memory address,0x02043b158f40
Stoichiometry,peagnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c PEA-glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDPglucose --> H + inner-core-oligosaccharide-lipid-A + UDP
GPR,NGO1353
Lower bound,0.0
Upper bound,999999.0


In [134]:
#SIALT1
r2 = model.reactions.get_by_id('SIALT1')
print(r2.check_mass_balance())
r2

{'C': -11.0, 'H': -18.0, 'N': -1.0, 'O': -8.0}


Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x02043b3ce640
Stoichiometry,"cmpacna_c + galgnagalicolipa_c --> cmp_c + lps_NM_c CMP-N-acetyl neuraminate + galatosyl-glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A --> CMP + mature lps, immunotype 3"
GPR,NGO1081
Lower bound,0.0
Upper bound,999999.0


In [135]:
#rename LPS_NM to LOS_gc

model.metabolites.get_by_id('lps_NM_c').id='los_GC_c'


In [136]:
model.metabolites.get_by_id('los_GC_c').name='los_GC_c'

In [137]:
#rename LPS_NM to LOS_GC
#Siaylation is not required for synthesis of LOS. Changed the "LOS" 
# designation to the the final unsialylated component. Renamed the sialylated LOS to "Sialylated LOS".
model.metabolites.get_by_id('los_GC_c').id='sialylated_los_GC_c'
model.metabolites.get_by_id('sialylated_los_GC_c').name='sialylated_los_GC_c'

model.metabolites.get_by_id('galgnagalicolipa_c').id='los_GC_c'
model.metabolites.get_by_id('los_GC_c').name='los_GC_c'

In [138]:
#Remove export of LOS and associated extracellular LOS components.
model.remove_reactions('LPS3tex')
model.remove_reactions('LPS8tex')

model.remove_metabolites(model.metabolites.get_by_id('galicolipa_e'))
model.remove_metabolites(model.metabolites.get_by_id('lps_NM_e'))

In [139]:
#For GLCTR1 - Formula for icolipa_c is incorrect. When glc is added to peagnahhlipa_c, the previous reaction did not add the oxygens. 
#The corresponding reactions in LOS synthesis all are also missing these 5 oxygens. This is corrected here.
#Used structure diagrams here PMID: 11496013 and here PMID: 30873172 to determine proper formulas. 

#The reaction GLCTR1 corresponds to a bigg ID for a slightly different reaction in E.coli LPS biosynthesis. 
#This was left in place, but may need to be changed at a later time to avoid confusion.

#Additionally for SIALT1 - LPS is sialylated, the NeuAC was not actually added to the structure for LPS.  
#This is corrected here. Note this sialylation is on the alpha chain of LOS only, which is more common. Not on the beta chain.


model.metabolites.get_by_id('icolipa_c').formula= 'C142H250N4O64P3'
model.metabolites.get_by_id('galicolipa_c').formula= 'C148H260N4O69P3'
model.metabolites.get_by_id('gnagalicolipa_c').formula= 'C159H275N4O89P4'
model.metabolites.get_by_id('los_GC_c').formula= 'C165H285N3O94P4'

model.metabolites.get_by_id('sialylated_los_GC_c').formula= 'C176H303N4O102P4'






print(r1)
print(r1.check_mass_balance())
print(r2)
print(r2.check_mass_balance())

GLCTR1: peagnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{}
SIALT1: cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c
{}


In [140]:
model.slim_optimize()

4.208806878287186

In [141]:
#Confirm new LOS reactions display correctly. 
model.reactions.get_by_id('SIALT1')

Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x02043b3ce640
Stoichiometry,cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c CMP-N-acetyl neuraminate + los_GC_c --> CMP + sialylated_los_GC_c
GPR,NGO1081
Lower bound,0.0
Upper bound,999999.0


In [142]:
#The more common isoform for Gc is 4HEXG. This has the addition of a glc on both the alpha and the beta chain.
#The NM model did not have this glc on the beta chain and instead has a PEA. The PEA and glc on the beta chain are mutually exclusive. 
# Added reaction GLCTR2 to add a glc to the beta chain. 

glcgnahhlipa_c= Metabolite(
    'glcgnahhlipa_c',
    formula='C132H232N3O61P2',
    name='M_glcgnahhlipa_c',
    compartment='cytosol')

udpg_c= model.metabolites.get_by_id('udpg_c')
udp_c = model.metabolites.get_by_id('udp_c')
h_c = model.metabolites.get_by_id('h_c')
gnahhlipa_c=model.metabolites.get_by_id('gnahhlipa_c')



GLCTR2 = cobra.Reaction('GLCTR2')
GLCTR2.notes = {}
GLCTR2.name = 'R_GLCTR2'
GLCTR2.gene_reaction_rule = 'NGO2072'
GLCTR2.lower_bound = 0.
GLCTR2.upper_bound = 1000.
GLCTR2.add_metabolites({
    gnahhlipa_c: -1.0,
    glcgnahhlipa_c: 1.0,
    udpg_c: -1.0,
    udp_c: 1.0,
    h_c: 1,
})

model.add_reactions([GLCTR2])
new_rxns += 1

r= model.reactions.get_by_id('GLCTR2')
print(r)
print(r.check_mass_balance())

GLCTR2: gnahhlipa_c + udpg_c --> glcgnahhlipa_c + h_c + udp_c
{}


In [143]:
#Remove reaction for addition of phosphoethanolamine to beta chain
model.remove_reactions('PEAT1')

In [144]:
#Add Gal to end of Beta chain to complete that segment of 4HEXG by adding reaction 
galglcgnahhlipa_c= Metabolite(
    'galglcgnahhlipa_c',
    formula='C138H242N3O66P2',
    name='M_galglcgnahhlipa_c',
    compartment='cytosol')

udpgal_c= model.metabolites.get_by_id('udpgal_c')
udp_c = model.metabolites.get_by_id('udp_c')
h_c = model.metabolites.get_by_id('h_c')
glcgnahhlipa_c=model.metabolites.get_by_id('glcgnahhlipa_c')



GALT3 = cobra.Reaction('GALT3')
GALT3.notes = {}
GALT3.name = 'R_GLCTR2'
GALT3.gene_reaction_rule = 'NGO2072'
GALT3.lower_bound = 0.
GALT3.upper_bound = 1000.
GALT3.add_metabolites({
    glcgnahhlipa_c: -1.0,
    galglcgnahhlipa_c: 1.0,
    udpg_c: -1.0,
    udp_c: 1.0,
    h_c: 1,
})

model.add_reactions([GALT3])
new_rxns += 1



In [145]:
r=model.reactions.get_by_id('GALT3')
print(r)
print(r.check_mass_balance())

GALT3: glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c
{}


In [146]:
#Create icolipa from the newly created 4HEXG Beta chain.
r = model.reactions.get_by_id('GLCTR1')

peagnahhlipa_c=model.metabolites.get_by_id('peagnahhlipa_c')

r.add_metabolites({
    peagnahhlipa_c: 1.0,
    galglcgnahhlipa_c: -1.0,
})


In [147]:
#Correct the icolipa formula and formulas for the alphachain components to reflect the new 4HEXG beta chain
model.metabolites.get_by_id('icolipa_c').formula= 'C144H252N3O71P2'   
model.metabolites.get_by_id('galicolipa_c').formula= 'C150H262N3O76P2'
model.metabolites.get_by_id('gnagalicolipa_c').formula= 'C158H276N4O81P2'
model.metabolites.get_by_id('los_GC_c').formula= 'C164H286N4O86P2'

model.metabolites.get_by_id('sialylated_los_GC_c').formula= 'C175H304N5O94P2'

print(r)
print(r.check_mass_balance())


GLCTR1: galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{}


In [148]:
#Check all the steps of LOS synthesis to ensure its balanced properly. 
#SIALT1
r = model.reactions.get_by_id('SIALT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x02043b3ce640
Stoichiometry,cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c CMP-N-acetyl neuraminate + los_GC_c --> CMP + sialylated_los_GC_c
GPR,NGO1081
Lower bound,0.0
Upper bound,999999.0


In [149]:
#GALT2
r = model.reactions.get_by_id('GALT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT2
Name,R_GALT2
Memory address,0x02043b13ffd0
Stoichiometry,gnagalicolipa_c + udpgal_c --> h_c + los_GC_c + udp_c glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDPgalactose --> H + los_GC_c + UDP
GPR,NGO2156
Lower bound,0.0
Upper bound,999999.0


In [150]:
#GNAT2
r = model.reactions.get_by_id('GNAT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT2
Name,R_GNAT2
Memory address,0x02043b18b9d0
Stoichiometry,"galicolipa_c + uacgam_c --> gnagalicolipa_c + udp_c galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + UDP-N-acetyl-D-glucosamine --> glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDP"
GPR,NGO2158
Lower bound,0.0
Upper bound,999999.0


In [151]:
#GALT1
r = model.reactions.get_by_id('GALT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT1
Name,R_GALT1
Memory address,0x02043b13fe50
Stoichiometry,"icolipa_c + udpgal_c --> galicolipa_c + h_c + udp_c inner-core-oligosaccharide-lipid-A + UDPgalactose --> galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + H + UDP"
GPR,NGO2159
Lower bound,0.0
Upper bound,999999.0


In [152]:
#GALT3
r = model.reactions.get_by_id('GALT3')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT3
Name,R_GLCTR2
Memory address,0x02043b2b3c70
Stoichiometry,glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c M_glcgnahhlipa_c + UDPglucose --> M_galglcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [153]:
#GLCTR2
r = model.reactions.get_by_id('GLCTR2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR2
Name,R_GLCTR2
Memory address,0x02043b2b38e0
Stoichiometry,gnahhlipa_c + udpg_c --> glcgnahhlipa_c + h_c + udp_c glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDPglucose --> M_glcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [154]:
#GLCTR1
r = model.reactions.get_by_id('GLCTR1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR1
Name,R_GLCTR1
Memory address,0x02043b158f40
Stoichiometry,galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c M_galglcgnahhlipa_c + UDPglucose --> H + inner-core-oligosaccharide-lipid-A + UDP
GPR,NGO1353
Lower bound,0.0
Upper bound,999999.0


In [155]:
#GNAT1
r = model.reactions.get_by_id('GNAT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT1
Name,R_GNAT1
Memory address,0x02043b18b910
Stoichiometry,hhlipa_c + uacgam_c --> gnahhlipa_c + udp_c heptosyl-heptosyl-kdo2-lipidA + UDP-N-acetyl-D-glucosamine --> glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDP
GPR,NGO1354
Lower bound,0.0
Upper bound,999999.0


In [156]:
#HEPT2
r = model.reactions.get_by_id('HEPT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT2
Name,R_HEPT2
Memory address,0x02043b1a3b80
Stoichiometry,adphep_LD_c + hlipa_c --> adp_c + h_c + hhlipa_c ADP-L-glycero-D-manno-heptose + heptosyl-kdo2-lipidA --> ADP + H + heptosyl-heptosyl-kdo2-lipidA
GPR,NGO0987
Lower bound,0.0
Upper bound,999999.0


In [157]:
#HEPT1
r = model.reactions.get_by_id('HEPT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT1
Name,R_HEPT1
Memory address,0x02043b1a3ac0
Stoichiometry,adphep_LD_c + lipa_c --> adp_c + h_c + hlipa_c ADP-L-glycero-D-manno-heptose + KDO-2-lipid-A --> ADP + H + heptosyl-kdo2-lipidA
GPR,NGO1934
Lower bound,0.0
Upper bound,999999.0


In [158]:
#All of LOS synthesis looks correct. Continue checking/ correcting other reactions. 

In [159]:
model.slim_optimize()

4.208806878287182

In [160]:
#HMT
r = model.reactions.get_by_id('HMT')
print(r.check_mass_balance())
r

{'H': -1.0}


Reaction identifier,HMT
Name,R_HMT
Memory address,0x02043b1b0e20
Stoichiometry,amet_c + his_L_c --> ahcys_c + mhis_c S-Adenosyl-L-methionine + L-Histidine --> S-Adenosyl-L-homocysteine + N(pi)-Methyl-L-histidine
GPR,NGO0746
Lower bound,0.0
Upper bound,999999.0


In [161]:
#No evidence that this reaction exists in Gc. 
#It forms a dead end metabolite that is not used elsewhere in the network.
#The gene (ngo0746) does possibly encode a methyltransferase, but this specific reaction doesn't appear to be found in any other models of bacteria, and isn't included in the CARVEME model for Gc. 
#As such, I am deleting the reaction. 

model.remove_reactions('HMT')

In [162]:
#MAc3
r = model.reactions.get_by_id('MAc3')
print(r.check_mass_balance())
r

{'O': -2.0}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x02043b1fb4c0
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534 and Orphan
Lower bound,0.0
Upper bound,999999.0


In [163]:
# Both Gc and Nm acylate their peptidoglycan. PMCID: PMC1231103
print(model.metabolites.get_by_id('murein5p4p_p').formula)
print(model.metabolites.get_by_id('murein5p4pOAc_p').formula)


C77H119N15O41
C79H123N15O41


In [164]:
#In the formula for murein5p4pOAc, the oxygen that gets added during acylation was forgotten. This is added here. 
model.metabolites.get_by_id('murein5p4pOAc_p').formula= 'C79H123N15O43'  


In [165]:
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x02043b1fb4c0
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534 and Orphan
Lower bound,0.0
Upper bound,999999.0


In [166]:
#peptidoglycan acylation has been shown to occur due to NGO0533 and NGO0534 (NMB1274 and NMB1273) aka PatA and PatB aka PacA and PacB. 
#THe third protein, proposed as an orphan, is apeI which actually is an esterase. 
#(PMID: 23373517)
#I correct the GPR rules for the MAc1 - MAc8 reactions here. 

NGO0533 =model.genes.get_by_id('NGO0533')
NGO0534= model.genes.get_by_id('NGO0534')
model.reactions.get_by_id('MAc1').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc2').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc3').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc4').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc5').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc6').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc7').gene_reaction_rule ='NGO0533 and NGO0534'
model.reactions.get_by_id('MAc8').gene_reaction_rule ='NGO0533 and NGO0534'
r

Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x02043b1fb4c0
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,999999.0


In [167]:
#Most of the PG in the cell wall is tetrapeptide in Gc PMCID: PMC551031 and is O-acytelated. 
#However tripeptide monomer gets released extracellularly and can cause immune responses through NOD1.
#Cleavage of the tetrapeptide to tripeptide, is driven by ldcA (NGO1274) (PMCID: PMC5646250)

#Confirmed, that tripeptide monomer is produced in the model (anhgm3p_p).



# Gc and neisseria in general use tetrapeptide PG in the cell wall, rather than tripeptide which is used in E.coli. 
# This model uses the "most common" o-acetylated tetrapeptide trimer to make PG for biomass. (murein4px4px4pOAc and murein 4p4p4pOAc).
#The model uses murein5p5p5p to make murein 4p4p4p (MDDCP6pp) which is then acetylated (MAc1).
#Its not clear how this would happen in nature since these reactions typically work on crosslinked peptidoglycan, not on uncrosslinked. 
#Additionally, peptidoglycan, incorporated into the cell wall, should be crosslinked. 

 
#As such, I am removing 4p4p4pOAC as an essential component of the cell wall for biomass (PEPGLY) and replacing it with 4px4px4p, since the cell wall consists of both acetylated and unmodified components.

r = model.reactions.get_by_id('PEPGLY')
print(r.check_mass_balance())
r

murein4p4p4pOAc_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
murein4px4px4p_p = model.metabolites.get_by_id('murein4px4px4p_p')

r.add_metabolites({
    murein4p4p4pOAc_p: 0.008334,
    murein4px4px4p_p : -0.008334,
})

print(r.check_mass_balance())
r






{'C': -1.5695700000000001, 'H': -2.433528, 'N': -0.24168599999999998, 'O': -0.8500679999999999}
{'C': -1.552902, 'H': -2.341854, 'N': -0.29169, 'O': -0.816732}


Reaction identifier,PEPGLY
Name,R_PEPGLY
Memory address,0x02043b2cde80
Stoichiometry,0.005556 murein4px4px4pOAc_p + 0.008334 murein4px4px4p_p <=> 0.1389 pepglycan_NM_p 0.005556 three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated + 0.008334 three-...
GPR,
Lower bound,-999999.0
Upper bound,999999.0


In [168]:
#I am deleting the reaction MDDCP6pp, which I cannot find evidence for currently. 
model.remove_reactions('MDDCP6pp')




In [169]:
#MAc7 and MAc8 which produces murein4p4p4pdOac and murein 4p4p4ptOAc respectively, where altered to use murein 4px4px4p as the base instead.
r = model.reactions.get_by_id('MAc7')
print(r.check_mass_balance())
r


murein4px4px4pdOAc_p= Metabolite(
    'murein4px4px4pdOAc_p',
    formula='C115H175N21O62',
    name='M_murein4px4px4pdOAc_p',
    compartment='periplasm')

murein4px4px4ptOAc_p= Metabolite(
    'murein4px4px4ptOAc_p',
    formula='C117H179N21O64',
    name='M_murein4px4px4ptOAc_p',
    compartment='periplasm')

murein4p4p4pOAC_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
murein4px4px4pOAc_p = model.metabolites.get_by_id('murein4px4px4pOAc_p')
murein4p4p4pdOAc_p=model.metabolites.get_by_id('murein4p4p4pdOAc_p')

murein4p4p4ptOAc_p=model.metabolites.get_by_id('murein4p4p4ptOAc_p')


r.add_metabolites({
    murein4p4p4pOAc_p: 1,
    murein4px4px4pOAc_p : -1,
    murein4p4p4pdOAc_p: -1,
    murein4px4px4pdOAc_p : 1,
})

print(r.check_mass_balance())
r



{}
{}


Reaction identifier,MAc7
Name,R_MAc7
Memory address,0x02043b1fb8e0
Stoichiometry,accoa_c + h2o_p + murein4px4px4pOAc_p --> coa_c + murein4px4px4pdOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated -->...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,999999.0


In [170]:
r = model.reactions.get_by_id('MAc8')
print(r.check_mass_balance())
r

r.add_metabolites({

    murein4p4p4pdOAc_p: 1,
    murein4px4px4pdOAc_p : -1,
    murein4p4p4ptOAc_p: -1,
    murein4px4px4ptOAc_p : 1,
})




{}


In [171]:
#And I am deleting MLTGY5pp as the lytic transglycosylase should also only work on crosslinked peptidoglycan, since its catalytic activity is to remove those crosslinks.
model.remove_reactions('MLTGY5pp')

In [ ]:
#Rename pepglycan metabolite
model.metabolites.pepglycan_NM_p.id="pepglycan_p"
model.metabolites.pepglycan_p.name='M_pepglycan_p'


In [172]:
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.000000,0.0
12DGR140tipp,0.000000,0.0
12DGR141tipp,0.000000,0.0
12DGR160tipp,0.000000,0.0
12DGR161tipp,0.000000,0.0
...,...,...
ZN2abcpp,0.013293,0.0
Zn2tex,0.013293,0.0
EX_apoACP_c_,-0.154685,0.0
GLCTR2,0.000000,0.0


In [173]:
#The third protein encoded in the operon is ApeI which de-O-acetylates and has broad acting esterase activity. 
#(PMID: 23373517)
#The original model removed the esterase activity to prevent futile cycling. 
#The reactions are kept the same here. 


In [174]:
model.slim_optimize()

4.20923925527708

In [175]:
r = model.reactions.get_by_id('NDPK9')
print(r.check_mass_balance())
r

{'H': -3.0}


Reaction identifier,NDPK9
Name,R_NDPK9
Memory address,0x02043b25d550
Stoichiometry,atp_c + didp_c <=> adp_c + ditp_c ATP + dIDP <=> ADP + dITP
GPR,NGO0597
Lower bound,-999999.0
Upper bound,999999.0


In [176]:
r.name = 'NADPK10'
r.id ='NADPK10'

model.metabolites.get_by_id('didp_c').formula= 'C10H11N4O10P2'

print(r.check_mass_balance())
r


{}


Reaction identifier,NADPK10
Name,NADPK10
Memory address,0x02043b25d550
Stoichiometry,atp_c + didp_c <=> adp_c + ditp_c ATP + dIDP <=> ADP + dITP
GPR,NGO0597
Lower bound,-999999.0
Upper bound,999999.0


In [ ]:
#No evidence for NGO1881 performing this reaction or for the mening homolog performing the reaction. 
model.reactions.NDPK1.gene_reaction_rule= 'NGO0597'
model.reactions.NDPK5.gene_reaction_rule= 'NGO0597'
model.reactions.NDPK8.gene_reaction_rule= 'NGO0597'

In [177]:
#NTD13
r = model.reactions.get_by_id('NTD13')
print(r.check_mass_balance())
r


{'H': 1.0}


Reaction identifier,NTD13
Name,R_NTD13
Memory address,0x02043b2813a0
Stoichiometry,h2o_c + nicrnt_c --> nicrns_c + pi_c H2O + Nicotinate-D-ribonucleotide --> Nicotinate D-ribonucleoside + Phosphate
GPR,NGO1058
Lower bound,0.0
Upper bound,999999.0


In [178]:
#changed nicrns_c formula based on Bigg metabolite formula. 
model.metabolites.get_by_id('nicrns_c').formula= 'C11H13NO6'

In [179]:
print(r.check_mass_balance())
r

{}


Reaction identifier,NTD13
Name,R_NTD13
Memory address,0x02043b2813a0
Stoichiometry,h2o_c + nicrnt_c --> nicrns_c + pi_c H2O + Nicotinate-D-ribonucleotide --> Nicotinate D-ribonucleoside + Phosphate
GPR,NGO1058
Lower bound,0.0
Upper bound,999999.0


In [180]:
#PHEMEFErel
r = model.reactions.get_by_id('PHEMEFErel')
print(r.check_mass_balance())
r

{'H': 4.0}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x02043b322880
Stoichiometry,3.0 h_c + 3.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 3.0 nadp_c 3.0 H + 3.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 3.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,999999.0


In [181]:
#Product of a simplified, 3 step reaction that requires 2 NADPHs. PMCID: PMC111422. 
#This reaction is described in kegg as requiring 3 "electron donors" and 3"electron acceptors, which is likely the cause of the error. https://www.genome.jp/entry/R11816


model.metabolites.get_by_id('biliverd_c').formula= 'C33H33N4O6'
NADPH_c =model.metabolites.get_by_id('nadph_c')
nadp_c =model.metabolites.get_by_id('nadp_c')

r.add_metabolites({
    NADPH_c: -3,
    nadp_c: 3.0
})

print(r.check_mass_balance())
r

{}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x02043b322880
Stoichiometry,3.0 h_c + 6.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 6.0 nadp_c 3.0 H + 6.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 6.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,999999.0


In [182]:
#PMCOAS
r = model.reactions.get_by_id('PMCOAS')
print(r.check_mass_balance())
r
#Okay to leave unbalanced. THis reaction is a simplification of a multistep ACP dependent reaction that isn't well understood. 

{'C': 7.0, 'H': 9.0, 'O': 3.0}


Reaction identifier,PMCOAS
Name,R_PMCOAS
Memory address,0x02043b353d00
Stoichiometry,coa_c --> pmcoa_c Coenzyme-A --> Pimeloyl-CoA
GPR,NGO1481 and NGO1725
Lower bound,0.0
Upper bound,999999.0


In [183]:
#SAMC
r = model.reactions.get_by_id('SAMC')
print(r.check_mass_balance())
r



{'H': 1.0}


Reaction identifier,SAMC
Name,R_SAMC
Memory address,0x02043b3b4e20
Stoichiometry,amet_c --> ametam_c + co2_c S-Adenosyl-L-methionine --> S-Adenosylmethioninamine + CO2
GPR,Orphan
Lower bound,0.0
Upper bound,999999.0


In [184]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SAMC
Name,R_SAMC
Memory address,0x02043b3b4e20
Stoichiometry,amet_c + h_c --> ametam_c + co2_c S-Adenosyl-L-methionine + H --> S-Adenosylmethioninamine + CO2
GPR,Orphan
Lower bound,0.0
Upper bound,999999.0


In [185]:
#SHSL2
r = model.reactions.get_by_id('SHSL2')
print(r.check_mass_balance())
r


{'H': -1.0}


Reaction identifier,SHSL2
Name,R_SHSL2
Memory address,0x02043b3c3e50
Stoichiometry,h2s_c + suchms_c --> hcys_L_c + succ_c Hydrogen-sulfide + O-Succinyl-L-homoserine --> L-Homocysteine + Succinate
GPR,NGO1149
Lower bound,0.0
Upper bound,999999.0


In [186]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL2
Name,R_SHSL2
Memory address,0x02043b3c3e50
Stoichiometry,h2s_c + suchms_c --> h_c + hcys_L_c + succ_c Hydrogen-sulfide + O-Succinyl-L-homoserine --> H + L-Homocysteine + Succinate
GPR,NGO1149
Lower bound,0.0
Upper bound,999999.0


In [187]:
#SHLS3
r = model.reactions.get_by_id('SHSL3')
print(r.check_mass_balance())
r


{'H': -1.0}


Reaction identifier,SHSL3
Name,R_SHSL3
Memory address,0x02043b3c3fa0
Stoichiometry,achms_c + cys_L_c --> ac_c + cyst_L_c O-acetyl-L-homoserine + L-Cysteine --> Acetate + L-Cystathionine
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [188]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL3
Name,R_SHSL3
Memory address,0x02043b3c3fa0
Stoichiometry,achms_c + cys_L_c --> ac_c + cyst_L_c + h_c O-acetyl-L-homoserine + L-Cysteine --> Acetate + L-Cystathionine + H
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [189]:
#SHLS5
r = model.reactions.get_by_id('SHSL5')
print(r.check_mass_balance())
r


{'H': -1.0}


Reaction identifier,SHSL5
Name,R_SHSL5
Memory address,0x02043b3ce460
Stoichiometry,h2o_c + suchms_c --> 2obut_c + nh4_c + succ_c H2O + O-Succinyl-L-homoserine --> 2-Oxobutanoate + Ammonium + Succinate
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [190]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL5
Name,R_SHSL5
Memory address,0x02043b3ce460
Stoichiometry,h2o_c + suchms_c --> 2obut_c + h_c + nh4_c + succ_c H2O + O-Succinyl-L-homoserine --> 2-Oxobutanoate + H + Ammonium + Succinate
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [191]:
#SHLS4
r = model.reactions.get_by_id('SHSL4')
print(r.check_mass_balance())
r


{'H': -2.0}


Reaction identifier,SHSL4
Name,R_SHSL4
Memory address,0x02043b3c32e0
Stoichiometry,achms_c + h_c + trdrd_c + tsul_c --> ac_c + hcys_L_c + so3_c + trdox_c O-acetyl-L-homoserine + H + Reduced-thioredoxin + Thiosulfate --> Acetate + L-Homocysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [192]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,SHSL4
Name,R_SHSL4
Memory address,0x02043b3c32e0
Stoichiometry,achms_c + trdrd_c + tsul_c --> ac_c + h_c + hcys_L_c + so3_c + trdox_c O-acetyl-L-homoserine + Reduced-thioredoxin + Thiosulfate --> Acetate + H + L-Homocysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0386
Lower bound,0.0
Upper bound,999999.0


In [193]:
#TFFE
r = model.reactions.get_by_id('TFFE')
print(r.check_mass_balance())
r


{'Fe': 1.0}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x02043b3e6b80
Stoichiometry,TfFe3_e --> Tf_e + 2.0 fe3_p Transferrin --> Transferrin residue + 2.0 Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and (Blank or NGO1377) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,999999.0


In [194]:
#Although transferrin holds two irons, Gc is only capable of stripping one off at a time. So transferrin is made to produce a single iron here. 
fe3_p =model.metabolites.get_by_id('fe3_p')

r.add_metabolites({
    fe3_p: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x02043b3e6b80
Stoichiometry,TfFe3_e --> Tf_e + fe3_p Transferrin --> Transferrin residue + Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and (Blank or NGO1377) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,999999.0


In [195]:
#TRDR1
#synthesis of thioredoxin. Protein based component.
r = model.reactions.get_by_id('TRDR1')
print(r.check_mass_balance())
r


{'X': 1.0}


Reaction identifier,TRDR1
Name,R_TRDR1
Memory address,0x02043b400f40
Stoichiometry,h_c + nadph_c --> nadp_c + trdrd_c H + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> Nicotinamide-adenine-dinucleotide-phosphate + Reduced-thioredoxin
GPR,NGO0580 and NGO2124
Lower bound,0.0
Upper bound,999999.0


In [196]:
trdox_c =model.metabolites.get_by_id('trdox_c')

r.add_metabolites({
    trdox_c: -1,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TRDR1
Name,R_TRDR1
Memory address,0x02043b400f40
Stoichiometry,h_c + nadph_c + trdox_c --> nadp_c + trdrd_c H + Nicotinamide-adenine-dinucleotide-phosphate--reduced + Oxidized-thioredoxin --> Nicotinamide-adenine-dinucleotide-phosphate + Reduced-thioredoxin
GPR,NGO0580 and NGO2124
Lower bound,0.0
Upper bound,999999.0


In [197]:
#Add thioredoxin import reaction. This is required to show "synthesis" of a protein based component. 
#Integration of Biomass Formulations of Genome-Scale Metabolic Models with Experimental Data Reveals Universally Essential Cofactors in Prokaryotes
#Joana C Xavier, Kiran Raosaheb Patil, Isabel Rocha
#PMID: 27939572 PMCID: PMC5249239

trdrd_c = model.metabolites.get_by_id('trdrd_c')

EX_trdrd_c_ = cobra.Reaction('EX_trdrd_c_')
EX_trdrd_c_.gene_reaction_rule = ''
EX_trdrd_c_.lower_bound = 0.
EX_trdrd_c_.upper_bound = 1000.
EX_trdrd_c_.add_metabolites({
    trdrd_c: -1.0,
})

model.add_reactions([EX_trdrd_c_])
new_rxns += 1


In [198]:
#TSULST
#synthesis of thioredoxin. Protein based component. 
r = model.reactions.get_by_id('TSULST')
print(r.check_mass_balance())
r


{'H': -2.0}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x02043b40cf10
Stoichiometry,acser_c + h_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + so3_c + trdox_c O-Acetyl-L-serine + H + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-999999.0
Upper bound,999999.0


In [199]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x02043b40cf10
Stoichiometry,acser_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + h_c + so3_c + trdox_c O-Acetyl-L-serine + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + H + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-999999.0
Upper bound,999999.0


In [201]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

12 12 12


,Reactions,Charge,Masses
0,3HAD161x,,"{'C': 29.0, 'H': 47.0, 'N': 2.0, 'O': 8.0, 'P'..."
1,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ASNSYN,,"{'H': 2.0, 'O': 2.0}"
3,BTS4,,{'S': 1.0}
4,CYTCS,,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
5,DASYN161a,,{'H': -1.0}
6,EAR161a,,"{'C': -29.0, 'H': -47.0, 'N': -2.0, 'O': -8.0,..."
7,DDPGA,,{'H': 2.0}
8,P3H5CD,,{'H': -1.0}
9,PGSA161a,,{'H': 1.0}


In [202]:
#DDPGA
r = model.reactions.get_by_id('DDPGA')
print(r.check_mass_balance())
r

{'H': 2.0}


Reaction identifier,DDPGA
Name,R_DDPGA
Memory address,0x02043b196e50
Stoichiometry,4h2oglt_c --> glx_c + 2.0 h_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + 2.0 H + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,999999.0


In [203]:
#There are two extra Hs produced in this reaction due to changes in protonation for h2oglt/glx. 
#Bigg does not indicate hydrogens are produced in this reaction. Correcting to match bigg. 
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: -2,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,DDPGA
Name,R_DDPGA
Memory address,0x02043b196e50
Stoichiometry,4h2oglt_c --> glx_c + pyr_c D-4-Hydroxy-2-oxoglutarate --> Glyoxylate + Pyruvate
GPR,NGO0713
Lower bound,0.0
Upper bound,999999.0


In [204]:
#P3H5CD
r = model.reactions.get_by_id('P3H5CD')
print(r.check_mass_balance())
r

{'H': -1.0}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x02043b299e50
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-999999.0
Upper bound,999999.0


In [205]:
#The protonation of e4hglu was changed when correcting HGTA. This results in having an extra h consumed in P3H5CD.
#This is corrected here. 
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 1.0,
})

print(r.check_mass_balance())
r

{}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x02043b299e50
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + 2.0 h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + 2.0 H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-999999.0
Upper bound,999999.0


In [ ]:
model.reactions.get_by_id('3HAD161x')

In [ ]:
model.metabolites.get_by_id('t3c5palmeACP_c').formula= 'C27H47N2O8PRS'

In [218]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

8 8 8


,Reactions,Charge,Masses
0,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ASNSYN,,"{'H': 2.0, 'O': 2.0}"
2,BTS4,,{'S': 1.0}
3,CYTCS,,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
4,DASYN161a,,{'H': -1.0}
5,PGSA161a,,{'H': 1.0}
6,PLIPA1G161a,,{'H': -1.0}
7,PMCOAS,,"{'C': 7.0, 'H': 9.0, 'O': 3.0}"


In [227]:
#Remaining imbalances are ok and due to proteinaceous components or reaction simplifications. 

In [228]:
complete(model)
print('Complete media', model.slim_optimize())

Complete media 4.209239255277078


In [229]:
len(model.genes)

515

In [231]:
#Subtract 2 genes for orphan and Sgene
model.id ='NGO_513'

In [232]:
model

Name,NGO_513
Memory address,0x020431c76280
Number of metabolites,1301
Number of reactions,1413
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular"


In [236]:
cobra.io.write_sbml_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_513.sbml')

In [238]:
cobra.io.save_json_model(model, cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_513.json')